In [52]:
from src.db.questions import QuestionsCollection
from src.models.clustering import QuestionClustering    

questions_collection = QuestionsCollection()
questions = questions_collection.fetch_all_questions()

data = questions_collection.preprocess_questions(raw_questions=questions)
data

2024-10-25 07:40:20:[INFO]:Connected to MongoDB database: codelab1
2024-10-25 07:40:20:[INFO]:Fetched 507 questions from the database.
2024-10-25 07:40:20:[INFO]:MongoDB connection closed.


,question_id,chapter,difficulty,concept
0,fc084299-5fff-4543-b5f6-5a7971b93da6,chuong-3,0.00,office
1,a6d25890-eecd-488a-8b89-2455786b3b80,chuong-3,0.00,office
2,3d9980e3-3d1c-4932-920e-f94a57340888,chuong-3,0.00,ppt
3,21b80886-9511-4cd4-bde3-1cf1878f21a2,chuong-3,0.00,office
4,239c4195-884a-49ea-80ea-45e18acbe1d4,chuong-3,0.00,office
...,...,...,...,...
502,1164b65d-1cba-8192-a242-e01d25e2bc9d,chuong-4,0.00,ham-nhap-xuat-du-lieu
503,1164b65d-1cba-81dc-b305-d078f5e25994,chuong-4,1.00,ham-nhap-xuat-du-lieu
504,1164b65d-1cba-81dd-911a-c2546a76ad11,chuong-4,0.00,ham-nhap-xuat-du-lieu
505,1164b65d-1cba-8165-93e4-c5ad83afc837,chuong-4,0.00,gioi-thieu-python


In [53]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
import numpy as np

In [54]:
import pandas as pd
df = pd.DataFrame(data)
concepts = df['concept'].apply(lambda x: x.split())
concepts

0                     [office]
1                     [office]
2                        [ppt]
3                     [office]
4                     [office]
                ...           
502    [ham-nhap-xuat-du-lieu]
503    [ham-nhap-xuat-du-lieu]
504    [ham-nhap-xuat-du-lieu]
505        [gioi-thieu-python]
506        [toan-tu-bieu-thuc]
Name: concept, Length: 507, dtype: object

In [55]:
w2v_model = Word2Vec(sentences=concepts, vector_size=10, window=3, min_count=1, sg=1)  # Size is 10-dimensional embedding
w2v_model.train(concepts, total_examples=w2v_model.corpus_count, epochs=10)

2024-10-25 07:40:20:[INFO]:collecting all words and their counts
2024-10-25 07:40:20:[INFO]:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-10-25 07:40:20:[INFO]:collected 31 word types from a corpus of 507 raw words and 507 sentences
2024-10-25 07:40:20:[INFO]:Creating a fresh vocabulary
2024-10-25 07:40:20:[INFO]:Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 31 unique words (100.00% of original 31, drops 0)', 'datetime': '2024-10-25T07:40:20.832393', 'gensim': '4.3.3', 'python': '3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'prepare_vocab'}
2024-10-25 07:40:20:[INFO]:Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 507 word corpus (100.00% of original 507, drops 0)', 'datetime': '2024-10-25T07:40:20.833437', 'gensim': '4.3.3', 'python': '3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]', 'platform': 'Windows-10-

(976, 5070)

In [56]:
def get_concept_embedding(concept):
    return w2v_model.wv[concept] if concept in w2v_model.wv else np.zeros(10)

In [57]:
df['concept_embedding'] = df['concept'].apply(lambda x: get_concept_embedding(x))
df

,question_id,chapter,difficulty,concept,concept_embedding
0,fc084299-5fff-4543-b5f6-5a7971b93da6,chuong-3,0.00,office,"[0.07311766, 0.05070262, 0.06757693, 0.0076286..."
1,a6d25890-eecd-488a-8b89-2455786b3b80,chuong-3,0.00,office,"[0.07311766, 0.05070262, 0.06757693, 0.0076286..."
2,3d9980e3-3d1c-4932-920e-f94a57340888,chuong-3,0.00,ppt,"[0.012188184, -0.084583245, -0.08223945, -0.00..."
3,21b80886-9511-4cd4-bde3-1cf1878f21a2,chuong-3,0.00,office,"[0.07311766, 0.05070262, 0.06757693, 0.0076286..."
4,239c4195-884a-49ea-80ea-45e18acbe1d4,chuong-3,0.00,office,"[0.07311766, 0.05070262, 0.06757693, 0.0076286..."
...,...,...,...,...,...
502,1164b65d-1cba-8192-a242-e01d25e2bc9d,chuong-4,0.00,ham-nhap-xuat-du-lieu,"[-0.08619688, 0.03665738, 0.051898837, 0.05741..."
503,1164b65d-1cba-81dc-b305-d078f5e25994,chuong-4,1.00,ham-nhap-xuat-du-lieu,"[-0.08619688, 0.03665738, 0.051898837, 0.05741..."
504,1164b65d-1cba-81dd-911a-c2546a76ad11,chuong-4,0.00,ham-nhap-xuat-du-lieu,"[-0.08619688, 0.03665738, 0.051898837, 0.05741..."
505,1164b65d-1cba-8165-93e4-c5ad83afc837,chuong-4,0.00,gioi-thieu-python,"[-0.08242677, 0.09299354, -0.0019766092, -0.01..."


In [58]:
pd.DataFrame(df['concept_embedding'].tolist(), index=df.index)

,0,1,2,3,4,5,6,7,8,9
0,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
1,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
2,0.012188,-0.084583,-0.082239,-0.002310,0.012373,-0.057434,-0.047253,-0.073461,0.083286,0.001213
3,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
4,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
...,...,...,...,...,...,...,...,...,...,...
502,-0.086197,0.036657,0.051899,0.057419,0.074669,-0.061677,0.011056,0.060473,-0.028401,-0.061735
503,-0.086197,0.036657,0.051899,0.057419,0.074669,-0.061677,0.011056,0.060473,-0.028401,-0.061735
504,-0.086197,0.036657,0.051899,0.057419,0.074669,-0.061677,0.011056,0.060473,-0.028401,-0.061735
505,-0.082427,0.092994,-0.001977,-0.019673,0.046036,-0.040953,0.027431,0.069400,0.060654,-0.075108


In [59]:
concept_embedding_df = pd.DataFrame(df['concept_embedding'].tolist(), index=df.index)
df = pd.concat([df, concept_embedding_df], axis=1)
df.drop(columns=['concept_embedding'], inplace=True)
df.columns = df.columns.astype(str)
df.head()

,question_id,chapter,difficulty,concept,0,1,2,3,4,5,6,7,8,9
0,fc084299-5fff-4543-b5f6-5a7971b93da6,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
1,a6d25890-eecd-488a-8b89-2455786b3b80,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
2,3d9980e3-3d1c-4932-920e-f94a57340888,chuong-3,0.00,ppt,0.012188,-0.084583,-0.082239,-0.002310,0.012373,-0.057434,-0.047253,-0.073461,0.083286,0.001213
3,21b80886-9511-4cd4-bde3-1cf1878f21a2,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
4,239c4195-884a-49ea-80ea-45e18acbe1d4,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361


In [62]:
label_encoder = LabelEncoder()
df['chapter_encoded'] = label_encoder.fit_transform(df['chapter'])

scaler = StandardScaler()
df['difficulty_scaled'] = scaler.fit_transform(df[['difficulty']])
df.head()

,question_id,chapter,difficulty,concept,0,1,2,3,4,5,6,7,8,9,chapter_encoded,difficulty_scaled
0,fc084299-5fff-4543-b5f6-5a7971b93da6,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112
1,a6d25890-eecd-488a-8b89-2455786b3b80,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112
2,3d9980e3-3d1c-4932-920e-f94a57340888,chuong-3,0.00,ppt,0.012188,-0.084583,-0.082239,-0.002310,0.012373,-0.057434,-0.047253,-0.073461,0.083286,0.001213,2,-0.693112
3,21b80886-9511-4cd4-bde3-1cf1878f21a2,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112
4,239c4195-884a-49ea-80ea-45e18acbe1d4,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112


In [65]:
concept_embedding_df.columns = concept_embedding_df.columns.astype(str)
embedding_columns = concept_embedding_df.columns
embedding_columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object')

In [67]:
X = df[['chapter_encoded', 'difficulty_scaled'] + list(embedding_columns)] 
X

,chapter_encoded,difficulty_scaled,0,1,2,3,4,5,6,7,8,9
0,2,-0.693112,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
1,2,-0.693112,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
2,2,-0.693112,0.012188,-0.084583,-0.082239,-0.002310,0.012373,-0.057434,-0.047253,-0.073461,0.083286,0.001213
3,2,-0.693112,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
4,2,-0.693112,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361
...,...,...,...,...,...,...,...,...,...,...,...,...
502,3,-0.693112,-0.086197,0.036657,0.051899,0.057419,0.074669,-0.061677,0.011056,0.060473,-0.028401,-0.061735
503,3,2.709695,-0.086197,0.036657,0.051899,0.057419,0.074669,-0.061677,0.011056,0.060473,-0.028401,-0.061735
504,3,-0.693112,-0.086197,0.036657,0.051899,0.057419,0.074669,-0.061677,0.011056,0.060473,-0.028401,-0.061735
505,3,-0.693112,-0.082427,0.092994,-0.001977,-0.019673,0.046036,-0.040953,0.027431,0.069400,0.060654,-0.075108


In [69]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from src.utils.logger import LOGGER

best_score = -1
max_clusters = 10
best_n_clusters = None
for n_clusters in range(2, max_clusters + 1):
    model = KMeans(n_clusters=n_clusters, random_state=42)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels)
    LOGGER.info(f"Number of clusters: {n_clusters}, Silhouette Score: {score:.4f}")

    if score > best_score:
        best_score = score
        best_n_clusters = n_clusters

kmeans = KMeans(n_clusters=best_n_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(X)

2024-10-25 07:48:35:[INFO]:Number of clusters: 2, Silhouette Score: 0.4988


2024-10-25 07:48:35:[INFO]:Number of clusters: 3, Silhouette Score: 0.5369
2024-10-25 07:48:35:[INFO]:Number of clusters: 4, Silhouette Score: 0.5881
2024-10-25 07:48:35:[INFO]:Number of clusters: 5, Silhouette Score: 0.6880
2024-10-25 07:48:36:[INFO]:Number of clusters: 6, Silhouette Score: 0.7243
2024-10-25 07:48:36:[INFO]:Number of clusters: 7, Silhouette Score: 0.7678
2024-10-25 07:48:36:[INFO]:Number of clusters: 8, Silhouette Score: 0.7829
2024-10-25 07:48:36:[INFO]:Number of clusters: 9, Silhouette Score: 0.6849
2024-10-25 07:48:36:[INFO]:Number of clusters: 10, Silhouette Score: 0.6884


In [71]:
df.head(20)

,question_id,chapter,difficulty,concept,0,1,2,3,4,5,6,7,8,9,chapter_encoded,difficulty_scaled,Cluster
0,fc084299-5fff-4543-b5f6-5a7971b93da6,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112,1
1,a6d25890-eecd-488a-8b89-2455786b3b80,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112,1
2,3d9980e3-3d1c-4932-920e-f94a57340888,chuong-3,0.00,ppt,0.012188,-0.084583,-0.082239,-0.002310,0.012373,-0.057434,-0.047253,-0.073461,0.083286,0.001213,2,-0.693112,1
3,21b80886-9511-4cd4-bde3-1cf1878f21a2,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112,1
4,239c4195-884a-49ea-80ea-45e18acbe1d4,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112,1
5,14ef74f3-2867-478b-a59f-8ae6bc9bc1d9,chuong-3,0.00,chen-doi-tuong,0.000946,0.030773,-0.068126,-0.013755,0.076686,0.073464,-0.036733,0.026427,-0.083171,0.062055,2,-0.693112,1
6,58e6e75e-5897-4a53-bec4-3954176b2422,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112,1
7,7f84856b-856b-4afc-9419-10b532139567,chuong-3,0.00,cong-thuc-ham,-0.037098,-0.087456,0.054375,0.065098,-0.007876,-0.067099,-0.070859,-0.024971,0.051433,-0.036652,2,-0.693112,1
8,8184ec54-399c-4336-a303-aa0f7dc958b3,chuong-3,0.00,office,0.073118,0.050703,0.067577,0.007629,0.063509,-0.034054,-0.009464,0.057686,-0.075216,-0.039361,2,-0.693112,1
9,299a269d-11c9-4ff5-ab2a-d541d3b55f29,chuong-3,0.00,cong-thuc-ham,-0.037098,-0.087456,0.054375,0.065098,-0.007876,-0.067099,-0.070859,-0.024971,0.051433,-0.036652,2,-0.693112,1
